In [1]:
import sys
sys.path.append('..')

In [2]:
import pandas as pd
from tqdm import tqdm
from faker import Faker

from utils.providers.CodenameProvider import CodenameProvider

In [3]:
fake = Faker(locale='pt_PT')

fake.add_provider(CodenameProvider)

In [4]:
def create_dataframe(x):
    clients = pd.read_csv('../data/generated_clients.csv', dtype=object)
    functions = pd.read_csv('../data/generated_exerts.csv', dtype=object)
    detectives = functions[functions['Funcao'] == '3']['Funcionario'].tolist()
    data = pd.DataFrame()

    for i in tqdm(range(x), desc='Creating DataFrame'):
        data.loc[i, 'Designacao'] = fake.codename()
        data.loc[i, 'DataInicio'] = fake.date_between(start_date='-1y', end_date='today')
        data.loc[i, 'DataTermino'] = fake.date_between(start_date='today', end_date='+1y') if fake.random_element([True, False]) else None
        data.loc[i, 'DataPagamento'] = fake.date_between(start_date=data.loc[i, 'DataTermino'], end_date='+1y') if (data.loc[i, 'DataTermino'] and fake.random_element([True, False])) else None
        data.loc[i, 'CustoAbertura'] = f"{fake.random.randint(1000, 99999)}.{fake.random.randint(0, 99)}"
        data.loc[i, 'Cliente'] = fake.random_element(clients['Id'])
        data.loc[i, 'Detetive'] = fake.random_element(detectives) if detectives else None

        # remove detective from list so he can't be assigned to another case
        detectives.remove(data.loc[i, 'Detetive']) if data.loc[i, 'Detetive'] else None

    return data

In [5]:
data = create_dataframe(20)
data.to_csv('../data/generated_cases.csv', index=True, index_label='Id')

Creating DataFrame: 100%|██████████| 20/20 [00:00<00:00, 1058.59it/s]
